In [1]:
from sqlalchemy import create_engine
import pandas as pd
import sqlite3 as sql
import re
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt


%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 100)
pd.options.display.max_colwidth = 3000

In [3]:
# clean = pd.read_excel('../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx', 0)
clean1 = pd.read_excel('../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx', 1)
clean2 = pd.read_excel('../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx', 2)
clean3 = pd.read_excel('../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx', 3)
# clean4 = pd.read_excel('../data/Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_033122.xlsx')

In [4]:
clean1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48020 entries, 0 to 48019
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Month          48020 non-null  object
 1   Precinct       48020 non-null  object
 2   Date           47926 non-null  object
 3   Created        47926 non-null  object
 4   Address        47926 non-null  object
 5   Incident Type  47926 non-null  object
 6   Priority       47926 non-null  object
dtypes: object(7)
memory usage: 2.6+ MB


In [5]:
clean1 = clean1.dropna()
clean1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47926 entries, 0 to 48019
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Month          47926 non-null  object
 1   Precinct       47926 non-null  object
 2   Date           47926 non-null  object
 3   Created        47926 non-null  object
 4   Address        47926 non-null  object
 5   Incident Type  47926 non-null  object
 6   Priority       47926 non-null  object
dtypes: object(7)
memory usage: 2.9+ MB


In [6]:
clean1.Precinct.value_counts()

H       7957
S       7953
C       6870
MT      5973
N       5515
M       4867
E       4408
W       4258
CW       118
PCW        4
CMD1       2
CMD2       1
Name: Precinct, dtype: int64

In [7]:
clean2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Month                 96 non-null     object
 1   Precinct              96 non-null     object
 2   ASSIST EMS            96 non-null     int64 
 3   DISORDERLY SUBJECT    96 non-null     int64 
 4   DOMESTIC DISTURBANCE  96 non-null     int64 
 5   INTOXICATED PERSON    96 non-null     int64 
 6   MENTAL CONDITION      96 non-null     int64 
 7   SUICIDAL PERSON       96 non-null     int64 
 8   SUSPICIOUS PERSON     96 non-null     int64 
 9   TRESPASSER            96 non-null     int64 
 10  WELFARE CHECK         96 non-null     int64 
dtypes: int64(9), object(2)
memory usage: 8.4+ KB


In [8]:
clean3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Month        96 non-null     object 
 1   Precinct     96 non-null     object 
 2   Priority-1   96 non-null     int64  
 3   Priority-2   96 non-null     int64  
 4   Priority-2L  96 non-null     int64  
 5   Priority-2M  96 non-null     int64  
 6   Priority-2N  96 non-null     int64  
 7   Priority-2T  96 non-null     int64  
 8   Priority-3   96 non-null     int64  
 9   Priority-3H  95 non-null     float64
 10  Priority-3M  96 non-null     int64  
 11  Priority-3Q  96 non-null     int64  
 12  Total        96 non-null     int64  
dtypes: float64(1), int64(10), object(2)
memory usage: 9.9+ KB


In [9]:
conn = sql.connect('../data/census.sqlite')
cur = conn.cursor()

In [10]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

In [11]:
available_tables = (cur.fetchall())

In [12]:
print(available_tables)

[('B01001',), ('B03002',), ('B11001',), ('B11002',), ('B15002',), ('B19001',), ('B19013',), ('B19301',), ('B25024',), ('B25075',), ('S1701',)]


In [13]:
cur.execute("SELECT * FROM B01001 LIMIT 5;")

In [14]:
B01001 = (cur.fetchall())

In [15]:
type(B01001)

list

In [16]:
B01001

[(0,
  '1400000US47037010103',
  'Census Tract 101.03, Davidson County, Tennessee',
  10103,
  37,
  47,
  'B01001_001E',
  2533,
  'Estimate!!Total:'),
 (1,
  '1400000US47037010104',
  'Census Tract 101.04, Davidson County, Tennessee',
  10104,
  37,
  47,
  'B01001_001E',
  2955,
  'Estimate!!Total:'),
 (2,
  '1400000US47037010105',
  'Census Tract 101.05, Davidson County, Tennessee',
  10105,
  37,
  47,
  'B01001_001E',
  4435,
  'Estimate!!Total:'),
 (3,
  '1400000US47037010106',
  'Census Tract 101.06, Davidson County, Tennessee',
  10106,
  37,
  47,
  'B01001_001E',
  3397,
  'Estimate!!Total:'),
 (4,
  '1400000US47037010201',
  'Census Tract 102.01, Davidson County, Tennessee',
  10201,
  37,
  47,
  'B01001_001E',
  4157,
  'Estimate!!Total:')]

In [17]:
pop_age_sex_df = pd.read_sql("SELECT * FROM B01001;", conn)

In [18]:
pop_age_sex_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B01001_001E,2533,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B01001_001E,2955,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B01001_001E,4435,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B01001_001E,3397,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B01001_001E,4157,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
7884,7884,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B01001_049E,53,Estimate!!Total:!!Female:!!85 years and over
7885,7885,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B01001_049E,0,Estimate!!Total:!!Female:!!85 years and over
7886,7886,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B01001_049E,68,Estimate!!Total:!!Female:!!85 years and over
7887,7887,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B01001_049E,0,Estimate!!Total:!!Female:!!85 years and over


In [19]:
cur.execute("SELECT * FROM B03002 LIMIT 5;")
B03002 = (cur.fetchall())
pop_race_ethnicity_df = pd.read_sql("SELECT * FROM B03002;", conn)
pop_race_ethnicity_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B03002_001E,2533,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B03002_001E,2955,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B03002_001E,4435,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B03002_001E,3397,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B03002_001E,4157,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
3376,3376,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B03002_021E,0,"Estimate!!Total:!!Hispanic or Latino:!!Two or more races:!!Two races excluding Some other race, and three or more races"
3377,3377,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B03002_021E,59,"Estimate!!Total:!!Hispanic or Latino:!!Two or more races:!!Two races excluding Some other race, and three or more races"
3378,3378,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B03002_021E,14,"Estimate!!Total:!!Hispanic or Latino:!!Two or more races:!!Two races excluding Some other race, and three or more races"
3379,3379,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B03002_021E,0,"Estimate!!Total:!!Hispanic or Latino:!!Two or more races:!!Two races excluding Some other race, and three or more races"


In [20]:
cur.execute("SELECT * FROM B11001 LIMIT 5;")
B11001 = (cur.fetchall())
households_type_df = pd.read_sql("SELECT * FROM B11001;", conn)
households_type_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B11001_001E,990,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B11001_001E,1221,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B11001_001E,1860,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B11001_001E,1152,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B11001_001E,1731,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
1444,1444,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B11001_009E,517,Estimate!!Total:!!Nonfamily households:!!Householder not living alone
1445,1445,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B11001_009E,807,Estimate!!Total:!!Nonfamily households:!!Householder not living alone
1446,1446,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B11001_009E,265,Estimate!!Total:!!Nonfamily households:!!Householder not living alone
1447,1447,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B11001_009E,0,Estimate!!Total:!!Nonfamily households:!!Householder not living alone


In [21]:
cur.execute("SELECT * FROM B11002 LIMIT 5;")
B11002 = (cur.fetchall())
pop_household_type_df = pd.read_sql("SELECT * FROM B11002;", conn)
pop_household_type_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B11002_001E,2533,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B11002_001E,2955,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B11002_001E,4430,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B11002_001E,3260,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B11002_001E,4157,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
1927,1927,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B11002_012E,2936,Estimate!!Total:!!In nonfamily households
1928,1928,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B11002_012E,5088,Estimate!!Total:!!In nonfamily households
1929,1929,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B11002_012E,1196,Estimate!!Total:!!In nonfamily households
1930,1930,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B11002_012E,0,Estimate!!Total:!!In nonfamily households


In [22]:
cur.execute("SELECT * FROM B15002 LIMIT 5;")
B15002 = (cur.fetchall())
sex_educational_attainment_pop_25_up_df = pd.read_sql("SELECT * FROM B15002;", conn)
sex_educational_attainment_pop_25_up_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B15002_001E,1872,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B15002_001E,2349,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B15002_001E,3621,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B15002_001E,2301,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B15002_001E,3163,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
5630,5630,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B15002_035E,157,Estimate!!Total:!!Female:!!Doctorate degree
5631,5631,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B15002_035E,58,Estimate!!Total:!!Female:!!Doctorate degree
5632,5632,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B15002_035E,5,Estimate!!Total:!!Female:!!Doctorate degree
5633,5633,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B15002_035E,0,Estimate!!Total:!!Female:!!Doctorate degree


In [23]:
cur.execute("SELECT * FROM B19001 LIMIT 5;")
B19001 = (cur.fetchall())
household_income_df = pd.read_sql("SELECT * FROM B19001;", conn)
household_income_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B19001_001E,990,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B19001_001E,1221,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B19001_001E,1860,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B19001_001E,1152,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B19001_001E,1731,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
2732,2732,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B19001_017E,311,"Estimate!!Total:!!$200,000 or more"
2733,2733,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B19001_017E,654,"Estimate!!Total:!!$200,000 or more"
2734,2734,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B19001_017E,99,"Estimate!!Total:!!$200,000 or more"
2735,2735,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B19001_017E,0,"Estimate!!Total:!!$200,000 or more"


In [24]:
cur.execute("SELECT * FROM B19013 LIMIT 5;")
B19013 = (cur.fetchall())
median_household_income_df = pd.read_sql("SELECT * FROM B19013;", conn)
median_household_income_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B19013_001E,70341,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B19013_001E,64844,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B19013_001E,64688,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B19013_001E,63864,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B19013_001E,62829,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)
...,...,...,...,...,...,...,...,...,...
156,156,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B19013_001E,81286,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)
157,157,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B19013_001E,75446,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)
158,158,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B19013_001E,59072,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)
159,159,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B19013_001E,-666666666,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)


In [25]:
cur.execute("SELECT * FROM B19301 LIMIT 5;")
B19301 = (cur.fetchall())
per_capita_income_df = pd.read_sql("SELECT * FROM B19301;", conn)
per_capita_income_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B19301_001E,33867,Estimate!!Per capita income in the past 12 months (in 2019 inflation-adjusted dollars)
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B19301_001E,38506,Estimate!!Per capita income in the past 12 months (in 2019 inflation-adjusted dollars)
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B19301_001E,35122,Estimate!!Per capita income in the past 12 months (in 2019 inflation-adjusted dollars)
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B19301_001E,30843,Estimate!!Per capita income in the past 12 months (in 2019 inflation-adjusted dollars)
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B19301_001E,34567,Estimate!!Per capita income in the past 12 months (in 2019 inflation-adjusted dollars)
...,...,...,...,...,...,...,...,...,...
156,156,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B19301_001E,58644,Estimate!!Per capita income in the past 12 months (in 2019 inflation-adjusted dollars)
157,157,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B19301_001E,66717,Estimate!!Per capita income in the past 12 months (in 2019 inflation-adjusted dollars)
158,158,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B19301_001E,30948,Estimate!!Per capita income in the past 12 months (in 2019 inflation-adjusted dollars)
159,159,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B19301_001E,-666666666,Estimate!!Per capita income in the past 12 months (in 2019 inflation-adjusted dollars)


In [26]:
cur.execute("SELECT * FROM B25024 LIMIT 5;")
B25024 = (cur.fetchall())
types_structure_df = pd.read_sql("SELECT * FROM B25024;", conn)
types_structure_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B25024_001E,1022,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B25024_001E,1382,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B25024_001E,1958,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B25024_001E,1186,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B25024_001E,1826,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
1766,1766,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B25024_011E,0,"Estimate!!Total:!!Boat, RV, van, etc."
1767,1767,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B25024_011E,0,"Estimate!!Total:!!Boat, RV, van, etc."
1768,1768,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B25024_011E,0,"Estimate!!Total:!!Boat, RV, van, etc."
1769,1769,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B25024_011E,0,"Estimate!!Total:!!Boat, RV, van, etc."


In [27]:
cur.execute("SELECT * FROM B25075 LIMIT 5;")
B25075 = (cur.fetchall())
value_owner_occupied_housing_units_df = pd.read_sql("SELECT * FROM B25075;", conn)
value_owner_occupied_housing_units_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B25075_001E,873,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B25075_001E,975,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B25075_001E,1626,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B25075_001E,885,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B25075_001E,1356,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
4342,4342,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B25075_027E,0,"Estimate!!Total:!!$2,000,000 or more"
4343,4343,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B25075_027E,0,"Estimate!!Total:!!$2,000,000 or more"
4344,4344,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B25075_027E,0,"Estimate!!Total:!!$2,000,000 or more"
4345,4345,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B25075_027E,0,"Estimate!!Total:!!$2,000,000 or more"


In [28]:
cur.execute("SELECT * FROM S1701 LIMIT 5;")
S1701 = (cur.fetchall())
poverty_status_last_12_months_df = pd.read_sql("SELECT * FROM S1701;", conn)
poverty_status_last_12_months_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037015631,"Census Tract 156.31, Davidson County, Tennessee",15631,37,47,S1701_C01_001E,12144.0,Estimate!!Total!!Population for whom poverty status is determined
1,1,1400000US47037015804,"Census Tract 158.04, Davidson County, Tennessee",15804,37,47,S1701_C01_001E,4093.0,Estimate!!Total!!Population for whom poverty status is determined
2,2,1400000US47037017701,"Census Tract 177.01, Davidson County, Tennessee",17701,37,47,S1701_C01_001E,2466.0,Estimate!!Total!!Population for whom poverty status is determined
3,3,1400000US47037017702,"Census Tract 177.02, Davidson County, Tennessee",17702,37,47,S1701_C01_001E,3511.0,Estimate!!Total!!Population for whom poverty status is determined
4,4,1400000US47037018301,"Census Tract 183.01, Davidson County, Tennessee",18301,37,47,S1701_C01_001E,8222.0,Estimate!!Total!!Population for whom poverty status is determined
...,...,...,...,...,...,...,...,...,...
29458,29458,1400000US47037015613,"Census Tract 156.13, Davidson County, Tennessee",15613,37,47,S1701_C03_061E,39.2,Estimate!!Percent below poverty level!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED!!Did not work
29459,29459,1400000US47037015620,"Census Tract 156.20, Davidson County, Tennessee",15620,37,47,S1701_C03_061E,58.5,Estimate!!Percent below poverty level!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED!!Did not work
29460,29460,1400000US47037016100,"Census Tract 161, Davidson County, Tennessee",16100,37,47,S1701_C03_061E,92.2,Estimate!!Percent below poverty level!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED!!Did not work
29461,29461,1400000US47037011400,"Census Tract 114, Davidson County, Tennessee",11400,37,47,S1701_C03_061E,36.0,Estimate!!Percent below poverty level!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED!!Did not work


In [29]:
clean1

,Month,Precinct,Date,Created,Address,Incident Type,Priority
0,July,C,07-01-2021,01:57:16,639 LAFAYETTE ST,INTOXICATED PERSON,2
1,July,C,07-01-2021,03:02:52,920 BROADWAY,WELFARE CHECK,2
2,July,C,07-01-2021,05:44:47,110 4TH AVE S,DISORDERLY SUBJECT,2
3,July,C,07-01-2021,05:49:07,800 4TH AVE S,SUSPICIOUS PERSON,2
4,July,C,07-01-2021,08:01:06,1033 DEMONBREUN ST,TRESPASSER,2
...,...,...,...,...,...,...,...
48015,March,W,03-31-2022,11:46:52,3811 HILLSBORO PIKE,TRESPASSER,2
48016,March,W,03-31-2022,12:24:06,7277 CHARLOTTE PIKE,SUICIDAL PERSON,2
48017,March,W,03-31-2022,17:59:25,MURPHY RD / 42ND AVE N,DISORDERLY SUBJECT,2
48018,March,W,03-31-2022,20:31:47,2828 KENWAY RD,SUSPICIOUS PERSON,2
